In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pylab as plt
import numpy as np

device = 'cuda:0' if torch.cuda.is_available() else "cpu"

def gen_data(N=100, d=10, low=0, high=10, target_idx=3):
    data = np.random.randint(low=low, high=high, size=(N,d))
    return data, data[:, target_idx]

N = 5000
low = 0 
high = 10
train_data, train_target = gen_data(N=N, low=low, high=high)
test_data, test_target = gen_data(N=N, low=low, high=high)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Recurrent Neural Networks (RNN)

An RNN will process a sequence of tokens. The pseudocode is something like the following:

token_list = [...]

hidden_vec = [0, ..., 0] #some fixed length or dimensionality

for token in token_list:

#lookup vector for each token from a hash table
    token_vec = embedding_table[token]
    
    #use previous hidden_vec and current token_vec to update hidden_vec
    #this is updating the state (hidden_vec) of the net using the new token
    
    hidden_vec = update(hidden_vec/previous state, token_vec/new data)
    
after the loop, hidden_vec encodes all the information about the input sequence and can be used to make a prediction

prediction = pred(hidden_vec)

for us, this could also be

prediction = pred(hidden_vec, index)

if the task is to predict the entry at a particular index

### Embedding

Suppose, we were working with natural text where the tokens were words. To feed in a word like "apple" to a neural network, we need to "numericalize" (i.e. convert it to a number) it.

The simplest solution is to map each unique token to a unique integer. For example:

"apple" -> 0

"is" -> 1

"a" -> 2

etc.

Note that the only requirement is that this mapping is one-to-one i.e. different words are mapped to different integers. The actual mapping, i.e. whether "apple" -> 0 or "apple" -> 59, doesn't matter.

Are there any problems with this encoding of tokens? One immediate problem is that it imposes an ordering on the tokens. "apple" is not less than "a" but 2 < 0. Depending on the machine learning model used, this ordered encoding can induce artifacts that are not real.

The solution to this problem is the so-called one-hot encoding. Suppose, there are N distinct words. Each word is mapped to a vector of size N where exactly one entry is 1 and the rest are zeros. Suppose, N = 3. Then,

"apple" -> [1,0,0]

"is" -> [0,1,0]

"a" -> [0,0,1]

Now, there is no order imposed on the tokens. Each vector is orthogonal to all other vectors (the dot product of vectors corresponding to distinct words is 0). This creates a couple of problems. If the number of tokens is large, the dimensionality N will also be large and this has implications for memory usage. Another problem is more conceptual. While each word is distinct (by definition) from every other word, words are not distinct by meaning. "apple" and "mango" are similar in the sense that they are both fruits but they are clearly also distinct (winter vs summer fruit etc.). Since vectors can be used to encode similarity, is it possible to map tokens to vectors such that (a) similar meaning words map to similar vectors and (b) dissimilar meaning words map to dissimilar vectors.

This is the problem embeddings solve. The philosophy in neural networks is to map each token to a unique vector is a relatively small (compared to the number of distinct tokens, N) dimensional (128 below) vector space. The embeddings are initialized randomly but are also adjusted during the learning process using the same exact process used to adjust/learn weights i.e. by computing derivatives and using gradient descent.

In [4]:
embedding_dim = 128
emb = nn.Embedding(num_embeddings=high-low, embedding_dim=128)

In [7]:
#can now look up embedding vectors based on input token (any value between low and high-1)

emb(torch.tensor(0))

tensor([-1.5423,  0.5266,  1.1882,  0.9355, -0.7791,  0.1069,  1.1231,  1.6374,
         0.9743, -0.0636, -0.9660, -0.9154,  0.8699, -0.0161, -0.0410,  1.1897,
        -0.4487, -0.0157,  0.3108,  0.1782, -1.2999,  0.4343,  0.0879, -0.3289,
        -0.3270,  2.1025,  0.3177,  1.0361, -1.3813, -2.7314, -0.8861,  0.8975,
        -0.6807, -0.0059,  1.8005, -0.3465, -0.2582,  0.1157, -1.1240,  1.2524,
        -0.8958,  0.4613, -0.9092,  0.6420,  0.2388, -0.3712, -0.2952,  1.2677,
        -0.7731,  0.8679, -0.1787,  0.5150,  0.4532, -1.6344,  1.3339,  0.9072,
         0.3769, -2.1367,  1.0948, -0.8444, -0.1842,  0.2181,  0.4649, -1.4071,
        -0.8272, -0.3051,  1.0011, -0.7690,  0.1095, -0.3177, -0.6874,  0.5006,
         0.5810,  1.7755, -0.2017, -0.6410,  0.4117,  0.4148,  2.4842, -0.9477,
        -0.2940,  1.2848, -1.1068, -0.3601,  0.1138,  0.6132,  0.2530, -1.5089,
         1.7862,  0.4336, -0.0694, -0.5198, -0.3226, -0.2274, -0.4900,  1.7660,
        -1.2420,  0.1898,  2.2897, -0.39

In [8]:
emb(torch.tensor(high-1))

tensor([ 0.2355,  0.6564, -1.9059,  2.4502,  1.7588, -0.5307,  0.7851, -0.6040,
        -1.4562, -0.4593,  2.6170,  0.4103,  2.0874, -0.9793,  0.8330,  1.4915,
        -1.3457,  0.5617, -0.6056,  0.6687, -0.5855,  1.0226, -1.8398, -0.1697,
         0.1374,  1.5272,  1.5056,  0.4490,  0.9732,  0.9570, -0.8251, -1.0611,
         0.3797,  0.3602, -0.1405, -0.5662, -1.0502,  1.2331,  1.0156,  0.1985,
        -1.8225,  0.0731,  0.8447,  0.3947,  2.4693, -2.5529,  0.8676, -1.1341,
         1.0207, -1.0664,  0.1900, -0.3369, -0.1403,  0.7167, -1.7084,  1.4050,
        -0.2450, -0.4552,  1.0088,  0.4901,  2.0871, -1.9398, -0.3584, -1.6027,
        -0.8999, -2.7576,  0.0708,  1.5830,  0.5663,  0.8460,  1.4456,  0.1762,
         1.0485,  1.1570,  0.4039, -1.2401,  0.8021,  0.4174,  1.3183, -0.0089,
         0.0594, -0.9420,  1.1330, -1.2755,  0.6854, -0.2580,  0.8663, -0.4360,
        -0.9288, -0.5829,  0.3857,  0.5792,  1.4026,  0.4559, -0.0683,  0.6871,
         0.9299,  1.0921, -1.8848, -0.17

In [9]:
#only values between low and high-1 have entries in the table
emb(torch.tensor(high))

IndexError: index out of range in self

### RNN definition

The code below defines the recurrent neural network. There are three broad classes of RNNs:

1. Vanilla RNNs - these tend to have a problem learning long-range behavior if the sequences are long. This is due to the so-called exploding and vanishing gradients problem (don't worry about this for now).

2. Long short-term memory networks (LSTMs): instead of having just one (hidden) state like RNNs, LSTMs maintain a long-term memory vector and a short-term memory vector. The coarse idea is to use the long-term memory vector to "remember" long-range patterns.

3. Gated Recurrent Units (GRUs): A simpler (less parameters/weights) form of LSTMs with the same underlying idea.

(We can go over the details in a call)

We will also use "attention". The core idea of attention is described below.

As an RNN processes an input sequence (sentence or byte vector), it generates a sequence of hidden vectors.

$$h_1, h_2, \ldots, h_T$$

where T = length of sequence.

The final hidden state is then used as a measure of context for any downstream tasks (predicting an output sequence or predicting a class for the sequence).

Attention refers to the idea that the context shouldn't consist just of $h_T$ but should be dynamic/flexible. 

To be finished - need to write more carefully

#### First, experiment with an LSTM to understand shapes of various tensors

In [20]:
#see: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
rnn = nn.LSTM(input_size = 128, #dimension of embedding
              hidden_size = 32, #
              num_layers = 2,
              batch_first = True, #expect (batch, seq, feature)
              dropout = 0.5,
              bidirectional=True                             
             )

In [47]:
#want to understand data flow. pick some input data
inp = torch.from_numpy(train_data[0:5])
inp.shape #(batch/sequence number, length/time)

torch.Size([5, 10])

In [48]:
emb(inp).shape #(batch/sequence number, length/time, embedding feature)

torch.Size([5, 10, 128])

In [49]:
print(type(rnn(emb(inp))))
len(rnn(emb(inp)))

<class 'tuple'>


2

In [50]:
print(type(rnn(emb(inp))[0]))
print(rnn(emb(inp))[0].shape) #(batch/sequence number, length/time, embedding feature*2 for bidirectional)

<class 'torch.Tensor'>
torch.Size([5, 10, 64])


In [51]:
print(type(rnn(emb(inp))[1]))
print(len(rnn(emb(inp))[1]))

<class 'tuple'>
2


In [52]:
print(type(rnn(emb(inp))[1][0]))
print(rnn(emb(inp))[1][0].shape)

<class 'torch.Tensor'>
torch.Size([4, 5, 32])


Why is the output above of shape (4,5,32)?

32 is the hidden dim i.e. the dimensionality of the hidden state and the cell state in an LSTM

5 is the number of sequences in the batch (if this is not convincing, try changing inp to have, say, 7 sequences)

where does the 4 come from? Claim: The 4 = 2 (num_layers) * 2 (bidirectional) cell states

Of course, we could have looked at the documentation:

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

Outputs: outputs, (h_n, c_n)

where 

outputs.shape = (number of examples, length of sequence, 2*hidden_dim)

h_n.shape = (2*num_layers, number of examples, hidden_dim)

c_n.shape = (2*num_layers, number of examples, hidden_dim)

In [39]:
print(type(rnn(emb(inp))[1][1]))
print(rnn(emb(inp))[1][1].shape)

<class 'torch.Tensor'>
torch.Size([4, 5, 32])


In [53]:
class Net(nn.Module):
    def __init__(self):
        super().__init__(low, high, emb_dim, hidden_size, num_layers=1)
        
        self.low = low
        self.high = high
        self.emb_dim = emb_dim
        
        self.emb = nn.Embedding(num_embeddings=high-low, embedding_dim=emb_dim)
        
        #see: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        self.rnn = nn.LSTM(input_size = embedding_dim,
                           hidden_size = hidden_size,
                           num_layers = num_layers,
                           batch_first = True, #expect (batch, seq, feature)
                           dropout = 0.5,
                           bidirectional=True                             
                          )
        
        self.pred = nn.Linear(2*hidden_size, high-low)
        
    def forward(self, x):
        out, (h_n, c_n) = self.rnn(x)
        
        #use the bidirectional hidden states from the last layer 

## Appendix

Seq2Seq (sequence to sequence) nets map an arbitrary length input sequence to an arbitrary length output sequence. If one wanted to map an arbitrary length input sequence to an output sequence of the same length, then one can use one RNN/LSTM.

### Seq2Seq paper: https://arxiv.org/pdf/1409.3215.pdf

**Core idea**:

* Use LSTM (encoder) with multiple layers ("deep") to map input sequence -> vector of fixed dimensionality.

* Use another LSTM (decoder) with multiple layers to generate output sequence starting from vector of fixed dimensionality produced by encoder.

* Additional trick that improves performance significantly: feed both the input sequence and reversed input sequence (bidirectional) to the encoder.

**Core idea figure**:

![Seq2Seq](seq2seq.png)

**Example applications where input/output values are sequences of varying lengths**:

* Speech recognition: audio -> text

* Machine translation: text -> text

* Question answering: text -> text

**How to think about the encoder-decoder architecture**:

Encoder: Read input tokens one at a time and generate compressed context vector of fixed dimensionality.

Decoder: Conditioned on context vector, generate sequences. In other words, all information about the input sequence is passed to the decoder through the context vector.

**Model architecture**:

* LSTMs instead of RNNs

* Two different LSTMs are used i.e. the same LSTM weights are not used for the encoder and the decoder

* Multi-layer LSTMs are used: 4 layers

* Order of tokens/words is reversed in input sequence i.e. instead of mapping: a,b,c -> $\alpha,\beta,\gamma$, map c,b,a -> $\alpha, \beta, \gamma$ so that a is closer to $\alpha$, b is closer to $\beta$. But note that now c is further away from $\gamma$

**Dataset details**: To get a sense of scale

Trained on 12 million sentences

Total unique French words: 384 million

Total unique English words: 304 million

For predictions, output space of words was restricted to 160k words in the source language (English) and 80k words in the target language (French). If word not in vocabulary, replace by special unknown token, UNK.

**Training details**:

loss = $\frac{1}{\mid \mathcal{S} \mid} \Sigma_{(T,S)\in\mathcal{S}} \log p(T|S)$

where $\mathcal{S}$ is the training set.

In other words, for each training example, the source sentence S is used to predict the target sentence T. For each token in the prediction, we know the actual/label token that should have been predicted. Compute the log probability for the label token and add them up over the target sentence tokens. Average these across each example in the training set.

* LSTMs: 4 layers, 1000 cells (hidden_dim), 1000 dimensional word embeddings (embedding_dim), input vocabulary size = 160k, output vocabulary size = 80k.

* LSTM weights ~ uniform(-0.08, 0.08)

* SGD without momentum, lr = 0.7 for first 5 epochs and halve lr every half epoch

* Total training time = 7.5 epochs

* Batch size = 128

* For each batch, compute $s = \Vert g \Vert_2 / 128$ where g = gradient. If $s > 5$, set $g = \frac{5g}{s}$ i.e. if length of vector exceeds 5, rescale to make length = 5.

* Most sentences are short (20-30 tokens) and some are long (>100 tokens). Make sure sentences in each batch are roughly the same length to avoid wasted computation.

* Use model parallelism with different layers on different GPUs. Note this paper was written before TensorFlow, PyTorch, Theano (?) etc.

**Prediction/inference details**:

Beam search with size = 1 or 2 works well. In detail, the decoder is used to predict a probability distribution over all possible tokens at each time-step. Since the output of the decode at time t is the input to the decoder at time t+1, we need a sample from the distribution. 

Ideally, we want the output sequence to be such that the sum of the log probabilities of the sampled tokens is maximized. This is not the same as a greedy sampling strategy. Beam search keeps track of the top k running sums of log probabilities.

**Questions**:

* While training, how should the hidden state of the encoder be initialized?
    * Zeros for each batch
    * Make hidden state initialization learned?
    * Keep hidden state evolving as batches get processed? This sounds troublesome since the implication is the order in which sequences are processed, matters.

### Seq2Seq + Attention paper: https://arxiv.org/pdf/1409.0473.pdf

**Core idea**:

* In the encoder-decoder setup, some tokens in the decoder/output sequence are far away from the corresponding tokens in the encoder/input sequence.

* Since all context about the input is compressed into the context vector, it might lose information about earlier tokens. In other words, the context vector can be an information bottleneck.

* This manifests itself as poor performance when the length of the input sequence gets long (longer than training data sequences).

* Can each token in the decoder be allowed to search for relevant tokens in the input sentence? This search has to be soft i.e. predict probabilities over the input sequence rather than hard choices.

* In this paper, a mechanism (attention) is introduced where, at each time-step in the decoder, a soft search over all hidden states in the encoder is carried out. This search is used to generate a new appropriate context vector that focuses on subsets of the input sequence. In other words, there is no need to encode the full input sequence into *one* context vector. Instead each input sequence is encoded into a sequence of vectors and during decoding, a soft search is carried out over this sequence of vectors.

**Note**: 

While in the previous paper, the final hidden state in the decoder, $h_T$ is used as a context vector, as this paper suggests, one could generalize to:

$$c = q({h_1, \ldots, h_T})$$

i.e. the context vector is some (fixed) function of all the hidden states.

**Core idea figure**:

![Model](seq2seq_attention.png)

The $x_t$ are the input tokens to the encoder. The $h_t$ are the hidden states (bidirectional). The key difference is the computation of the decoder's hidden state at time t, $s_t$.

In the classic encoder-decoder picture, $s_t = f(s_{t-1}, y_{t-1})$. In this model,

$$s_t = f(s_{t-1}, y_{t-1}, h_{1}, h_{2}, \ldots, h_T)$$

where $f()$ sloppily refers to "some function" (not the same one in both computations). So, now the computation has direct access to each encoder hidden state.

**Precise formulation**:

Given an input sequence S, of length T, the encoder produces hidden states, $h_1, \ldots, h_T$.

These hidden states are used to compute a context vector, $c$. In the seq2seq paper, $c = h_T$.

The decoder conditions on the context vector i.e. it intializes its hidden state, $s_0$ so that $s_0 = c$.


The first token is a special SOS (start of sentence) token and is passed as the first input $y_0$. The decoder hidden state is updated:

$$s_1 = f(s_0, y_0)$$

which is used to compute the probability distribution over the vocabulary:

$$p(y_1\mid y_0, c) = g(s_1)$$


For any time t,

$$\boxed{s_t = f(s_{t-1}, y_{t-1})}$$

and $$\boxed{p(y_t\mid y_0,\ldots, y_{t-1}, c) = g(s_{t})}$$

Both of these equations implictly depend on the context vector $c$ since every calculation depends on $s_0 = c$. We make this explicit:

$$\boxed{s_t = f(s_{t-1}, y_{t-1}, c)}$$

and $$\boxed{p(y_t\mid y_0,\ldots, y_{t-1}, c) = g(s_{t}, c)}$$

One way of looking at attention is to make $c$ dependent on time t, i.e.:

$$\boxed{s_t = f(s_{t-1}, y_{t-1}, c_t)}$$

and $$\boxed{p(y_t\mid y_0,\ldots, y_{t-1}) = g(s_{t}, c_t)}$$


This implies an order of computation:

* Compute $c_t$ which is the context vector at time t.

* Compute hidden state $s_t$ from $c_t, s_{t-1}, y_{t-1}$.

* Compute prob distribution from $s_t, c_t$.

The context vector, instead of being just $h_T$ (last hidden vector in the encoder) is now generalized to a linear (convex) combination of all encoder hidden vectors:

$$c_t = \Sigma_{j} \alpha_{tj}h_j$$

where $\alpha_{tj}$ can be interpreted as probabilities as shown below.

$$\alpha_{tj} = \frac{e^{e_{tj}}}{\Sigma_{k}e^{e_{tk}}}$$

This is essentially a Boltzmann probability or you can think of $e_{tj}$ as the energy of the $j$th configuration, the exponential ensures that $\alpha_{tj} > 0$ and the denominator ensures the $\alpha_{tj}$ add up to 1 (when summed over $j$). 

The next question is how $e_{tj}$ is computed and there are many choices here. In general, $e_{tj} = a(s_{t-1}, h_j)$. Note that $e_{tj}$ is computed before $s_t$ and hence depends on $s_{t-1}$ and not $s_t$. In words, "based on what the decoder has generated so far till time t-1 and the summary encoded in $s_{t-1}$, what information can be gathered from the encoder's hidden states to compute the next hidden state $s_t$ so the next token can be computed".

**Open question**: what would happen if $s_t$ were used to compute $c_t$?


**Model architecture**:

Encoder: Bidirectional RNN (LSTM etc.)

Decoder: RNN (LSTM etc.) + Attention as described above

Loss: multi-class log loss/cross-entropy

Decoding strategy: Beam search

Optimizer: Adadelta (training time for paper's model ~ 5 days on what hardware?)